In [24]:
import requests
import pandas as pd
from tqdm import tqdm
from datetime import date

community = "SluttyConfessions"
URL = f"https://www.reddit.com/r/{community}/new.json"
all_posts = []
after = None
headers = {
	"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
}

for i in tqdm(range(1000//100)):
	params = {'limit': 100, 'after': after}
	response = requests.get(URL, headers=headers, params=params)
	if response.status_code == 200:
		data = response.json()
		all_posts += data['data']['children']
		after = data['data']['after']
		if after is None:
			break
	else:
		print(f"Failed to get data. Status code: {response.status_code}")

dic = {
	"titles":[post['data']['title'] for post in all_posts],
	"text":[post['data']['selftext'] for post in all_posts],
	"num_comments": [post['data']['ups'] for post in all_posts],
	"upvote_ratio":[post['data']['upvote_ratio'] for post in all_posts],
	"ups":[post['data']['num_comments'] for post in all_posts],
	"num_crossposts":[post['data']['num_crossposts'] for post in all_posts],
	"created_utc":[post['data']['created_utc'] for post in all_posts],

}
data = pd.DataFrame(dic)
data['comments_per_upvotes'] = data['num_comments'] / data['ups']
data["created_at"] = pd.to_datetime(data["created_utc"], unit='s')
data["days_since_created"] = (pd.to_datetime(date.today()) - pd.to_datetime(data["created_utc"], unit='s')).dt.days
data["days_since_created"] = data["days_since_created"].mask(data["days_since_created"] < 0, 1)
data = data.sort_values(by=['num_comments', 'comments_per_upvotes','upvote_ratio', 'ups'], ascending=[False ,False, False, False])

data.to_csv(f"data_{community}.csv", mode="w", encoding='utf-8-sig', index=False)
print(f"CSV file exported successfully - contains {len(data)} lines")

  0%|          | 0/10 [00:00<?, ?it/s]

 90%|█████████ | 9/10 [00:22<00:02,  2.47s/it]

CSV file exported successfully - contains 982 lines


In [25]:
data

,titles,text,num_comments,upvote_ratio,ups,num_crossposts,created_utc,comments_per_upvotes,created_at,days_since_created
776,I (36F) had anal sex with my (27M) neighbor to...,So my son is getting to the age where he reall...,2716,0.98,216,0,1.766686e+09,12.574074,2025-12-25 18:11:20,2
850,I didn’t realize I was flashing my students du...,As I entered the lecture hall for my 9:00 am c...,1857,0.98,75,2,1.766630e+09,24.760000,2025-12-25 02:35:03,2
569,I fucked her friend the morning after our thre...,Waking up the next morning was a trip. We were...,1806,0.98,52,0,1.766775e+09,34.730769,2025-12-26 18:52:55,1
644,I completely stripped naked in front of my bf ...,Three of us went on a hike and it was pretty d...,1338,0.96,58,0,1.766746e+09,23.068966,2025-12-26 10:41:34,1
362,Husband paid for me to do a boudoir shoot for ...,"One of the gifts my Husband, 38m, got me, 23f,...",1239,0.91,107,0,1.766851e+09,11.579439,2025-12-27 15:54:13,0
...,...,...,...,...,...,...,...,...,...,...
103,24M looking to be used by a woman elder than him,"Hey there, so one of my biggest unexplored kin...",0,0.25,1,0,1.766938e+09,0.000000,2025-12-28 16:05:15,1
79,I sneak piss into womens holes and they never ...,every once in awhile depending on the type of ...,0,0.17,11,0,1.766943e+09,0.000000,2025-12-28 17:26:20,1
250,The first time I (22m) saw my gf (22f) giving ...,I remember the evening my girlfriend cucked me...,0,0.50,0,0,1.766884e+09,NaN,2025-12-28 01:14:32,1
962,I’ve had a fantasy behind my girlfriends back ...,"Sooooooo, this is a super long story but basic...",0,0.50,0,0,1.766564e+09,NaN,2025-12-24 08:06:34,3


In [26]:
from datetime import date

print(pd.to_datetime(date.today()))


2025-12-28 00:00:00
